In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath('.')))

# Essential imports
import pandas as pd
import sqlalchemy
from datetime import datetime, date, timedelta
import requests
import json
import time
from tqdm import tqdm

# Local imports
from src.database import get_database_connection, get_api_key

print("🚀 Phase 1E: Historical News Collection - Ready!")
print("📊 Mission: Build real historical news pipeline for strategy validation")


In [ ]:
# Create decoupled database schema
def create_decoupled_schema():
    """Create new tables for decoupled news collection and sentiment analysis"""
    
    schema_sql = """
    -- Raw news articles storage
    CREATE TABLE IF NOT EXISTS raw_news_articles (
        id SERIAL PRIMARY KEY,
        symbol_id INTEGER REFERENCES symbols(id),
        article_date DATE NOT NULL,
        title TEXT NOT NULL,
        content TEXT,
        summary TEXT,
        source VARCHAR(100),
        url TEXT,
        published_at TIMESTAMP,
        relevance_score DECIMAL(3,2),
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        UNIQUE(url, symbol_id)
    );
    
    -- Processed sentiment results
    CREATE TABLE IF NOT EXISTS processed_sentiment (
        id SERIAL PRIMARY KEY,
        symbol_id INTEGER REFERENCES symbols(id),
        analysis_date DATE NOT NULL,
        smo_score DECIMAL(3,2),
        smd_score DECIMAL(3,2),
        smc_score DECIMAL(3,2),
        sms_score DECIMAL(3,2),
        sdc_score DECIMAL(3,2),
        articles_analyzed INTEGER,
        confidence_score DECIMAL(3,2),
        analysis_summary TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        UNIQUE(symbol_id, analysis_date)
    );
    
    -- Create indexes
    CREATE INDEX IF NOT EXISTS idx_raw_news_symbol_date ON raw_news_articles(symbol_id, article_date);
    CREATE INDEX IF NOT EXISTS idx_processed_sentiment_date ON processed_sentiment(symbol_id, analysis_date);
    """
    
    try:
        engine = get_database_connection()
        with engine.connect() as conn:
            conn.execute(sqlalchemy.text(schema_sql))
            conn.commit()
        
        print("✅ Decoupled database schema created successfully")
        return True
    except Exception as e:
        print(f"❌ Error creating schema: {e}")
        return False

# Create the new schema
create_decoupled_schema()
